![img](utils/usak.webp)

# ¿Es determinante el entorno  para los asesinos en serie?
##### Memoria del proyecto

## Índice
1. [Introducción](#introducción)
2. [Hipótesis](#hipótesis)
3. [Limpieza de datos](#limpieza-de-datos)
4. [Análisis de los datos y comprobación de hipótesis](#Análisis-de-los-datos-y-comprobación-de-hipótesis)
    
    4.1. Países en los que están más presentes o cometen más crímenes

    4.2. Clase socioeconómica

    4.3. Densidad de población

    4.4. ¿Realmente son más comunes en América y Europa?

    4.5. ¿En qué países les descubren antes?
5. [Visualización de los datos](#visualizaciones)
6. [Conclusiones y consideraciones](#conclusiones)

## Introducción
Un asesino en serie no comete crímenes pasionales, ni suele elegir a sus víctimas por rencor ni para ajustar cuentas. De hecho, puede llegar a ser metódico y tener templanza.
Entonces, ¿cómo surge este perfil de asesino? ¿Qué factores los motiva a actuar?

Este análisis de datos exploratorio se centrará en el estudio de múltiples asesinos en serie para determinar patrones que puedan desencadenar sus conductas. Sobre todo, los factores en los que se centrará son:

+ País de procedencia
+ Número de víctimas
+ Densidad de población
+ Índice de pobreza

#### Fuentes de datos

Para este proyecto he utilizado [cuatro datasets de Kaggle](https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset/data), separados según el número de víctimas de los asesinos, un [JSON de la densidad de población de cada país](https://data.world/samayo/country-names/workspace/file?filename=country-population-density.json), el [índice de pobreza de cada país según OECD Data](https://data.oecd.org/inequality/poverty-rate.htm) y, por último, una [clave de códigos ISO y nombres de países](https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv).

Con los datos recopilados, dispongo de información de 355 asesinos, 6292 víctimas, 57 países, 5 continentes y datos que van de 1900 a 2018.

## Hipótesis
1. ¿Hay ciertos **países** en los que estos asesinos están más presentes o donde cometen más crímenes?

2. Suelen pertenecer a **clases socioeconómicas bajas** o es lo que predomina a su alrededor

3. Ese tipo de perfil de asesino tiende a aparecer en lugares con **alta densidad de población**

4. En las series siempre se habla de los casos de estadounidenses, pero ¿realmente son más comunes en **América y Europa** o es solo que en los otros continentes no se estudian estos casos o no se publica esta información?
(Claramente partimos del sesgo de nuestra cultura occidental y del peso que tiene EE.UU. en esta).

5. ¿En qué países les descubren o capturan antes?

## Limpieza de datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
hivict = pd.read_csv("..\\data\\Highest_victim_count.csv") #highest victim count
popdens = pd.read_json("..\\data\\country-population-density.json") #population density
povrates = pd.read_csv('..\\data\\spoverty_rate.csv') #poverty rate
mainvict = pd.read_csv("..\\data\\15_to_30_victim_count.csv") #main, 15-30 victim count
less5vict=pd.read_csv("..\\data\\Lessthan_5_victim_count.csv")
fr5to14vict=pd.read_csv("..\\data\\5_to_14_victim_count.csv")
countrycode=pd.read_csv("..\\data\\iso_countrycode.csv")

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\Highest_victim_count.csv'

#### Limpieza de los cuatro datasets de los sujetos
(Este proceso desarrollado se encuentra en el *notebook* [1datacleaning.ipynb](\\notebooks\\1datacleaning.ipynb).)

+ El primer paso, y el más laborioso, fue el de limpiar los datos.
+ La columna de países de procedencia de los sujetos incluía varios valores en la misma celda (además de datos como "Brasil (supuestamente)"), por lo que he tenido que limpiarla separando los valores en una lista (```str.split()```) y dividiéndolos en filas con ``` pd.explode()```. De esta forma, si un sujeto perpetró crímenes en tres países, aparecerá tres veces, ya que lo que nos importa realmente son los países en los que tienen lugar dichos crímenes.

In [ ]:
#separating countries in the same cell as a list
hivict['country']=hivict['country'].str.split('\r\n')

#dividing list in several rows and removing the " (alleged)" bit
hivictexp=hivict.explode('country')
hivictexp['country']=hivictexp['country'].str.replace(" (alleged)","")
hivictexp['country'].unique()

Este proceso se repite para las cuatro tablas de datos de asesinos, que posteriormente se concatenarán.

#### Limpieza de los datasets de datos demográficos y la clave de país-código ISO

(Este proceso desarrollado se encuentra en el notebook [1datacleaning.ipynb](\\notebooks\\1datacleaning.ipynb).)

+ Lo primero que tuve que hacer fue homogeinizar los nombres de las columnas y eliminar las que eran irrelevantes:

In [ ]:
#añadir head

In [ ]:
# homogeneizar nombres de columnas
for col in povrates:
    povrates.rename(columns={col:col.lower()}, inplace=True)

povrates.rename(columns={'location':'code', 'time':'year'}, inplace=True)

povrates=povrates[['code', 'year','value']]
povrates.head()

+ Principalmente quería basarme en dos datos demográficos: **la densidad de población y el índice de pobreza**.Sin embargo, el dataset de *poverty rates* incluía el código ISO de los países en lugar de su nombre, lo que me obligó a buscar otro csv que me permitiera hacer la correspondencia entre ambos.


In [ ]:
countrycode=countrycode[['English short name lower case', 'Alpha-3 code']]
countrycode.rename(columns={'English short name lower case':'country name', 'Alpha-3 code': 'code'},inplace=True)
countrycode.sort_values(['code'])

+ Después tuve que limpiar el dataset de índice de pobreza, que tenía varios valores según el año, pero muchos NaN también. **Decidí agruparlos por país y hacer una media del índice de pobreza, para dejar solo una fila por país, que después sería una nueva columna del dataframe.**

In [ ]:
avgpovratecountry=povrates.groupby('code')['value'].mean()
avgpovratecountry=pd.DataFrame(avgpovratecountry).reset_index()
avgpovratecountry.rename(columns={'value':'mean value'},inplace=True)
avgpovratecountry.drop(columns=['level_0','index'],inplace=True)

+ Una vez hecho esto, se podía eliminar la columna de año y los valores nulos.

In [ ]:
povrates.dropna(subset=['value'],inplace=True)
povrates.dropna(subset=['year'],inplace=True)
simplepovrates=povrates.drop(columns='year').drop_duplicates('code')

+ A partir de ahí, tuve que empezar a utilizar ```pd.merge()``` para juntar los datos demográficos y la clave de país-código ISO.

In [ ]:
cleanpovrate=simplepovrates.merge(avgpovratecountry, on='code').drop(columns='value')

+ Se añade el nombre de los países a *pov rates* haciendo un merge con la anterior tabla.

In [ ]:
countrypovrate=pd.merge(cleanpovrate, countrycode, how='outer')
countrypovrate.dropna(subset=['mean value'],inplace=True)

#### Homogeinizar los nombres de los países en los distintos datasets


+ **Ahí me di cuenta de que algunos países del dataset de asesinos no estaban escritos igual que los de este dataset** (por ejemplo, 'Korea, Republic of', frente a 'South Korea').
+ Esto me obligó a crear un diccionario para corregir y homogeninizar los nombres de los países:

In [ ]:
wrongpovratecountries={'Korea, Republic of':'South Korea'}

for wrong,right in wrongpovratecountries.items():
    countrypovrate['country name']=countrypovrate['country name'].str.replace(wrong, right)

+ El mismo procedimiento se repitió para el dataest de la densidad de población:

In [ ]:
wrongpopdenscountries={'Iran, Islamic Rep.':'Iran','Russian Federation':'Russia','Korea, Rep.':'South Korea',
                       'Venezuela, RB':'Venezuela','Yemen, Rep.':'Yemen'}

for wrong,right in wrongpovratecountries.items():
    popdens['country']=popdens['country'].str.replace(wrong, right)

(Este proceso desarrollado se encuentra en el notebook [2datacleaning_matching.ipynb](\\notebooks\\2datacleaning_matching.ipynb).)

+ Se concatenan los cuatro datasets de datos de asesinos con ```pd.concat()``` y se ve los países cuyo nombre no encaja con los otros datasets (para hacer otro diccionario).

In [ ]:
#keys are wrong, values are the correct ones (in killer df)
wrongkillercountries={'Allied-occupied Germany':'Germany','Austria-Hungary':'Hungary',
                      'Czechoslovakia':'Czech Republic','East Germany':'Germany','German Empire':'Germany',
                      'Kingdom of Romania':'Romania','Ottoman Empire':'Iran','Portuguese Angola':'Angola',
                      'West Germany':'Germany', 'Soviet Union':'Ukraine', 'Yugoslavia':'Serbia'}

for wrong,right in wrongkillercountries.items():
    totalvict['country']=totalvict['country'].str.replace(wrong, right)

(Este proceso desarrollado se encuentra en el *notebook* [3mergingdf.ipynb](\\notebooks\\3mergingdf.ipynb).)

+ Por último, en un último Jupyter notebook de limpieza se fusionan todos los datasets para poder trabajar con un dataframe definitivo y se eliminan las columnas innecesarias.

In [ ]:
#merged killers df with poverty rate df
kill_povrate=killers.merge(povrate, how='left')

kill_povrate['mean value'].isnull().value_counts() #quite a few nans for poverty rate

#remove "code" column
kill_povrate.drop(columns='code', inplace=True)

In [ ]:
#merged killers_povrate df with pop density df
dfkillers=kill_povrate.merge(popdens, how='left')
dfkillers.rename(columns={'mean value':'pov rate'},inplace=True)

dfkillers.to_csv("..\\data\\cleaned\\eda_df.csv")

+ El *eda_df.csv* ya está listo para el análisis. Veamos el resultado final:

In [ ]:
len(dfkillers[(dfkillers['density'].notnull()) & (dfkillers['pov rate'].notnull())])
#201 killers with both data
#pov rate: 205 values, 150 nan
#density: 313 values, 42 nan

+ Hay 150 países sin datos de índice de pobreza y 42 países sin datos de densidad de población. **Es decir, disponemos de todos los datos de 201 sujetos**.
+ Se exporta otro csv sin NaN también:

In [ ]:
notnullkillers=dfkillers[(dfkillers['density'].notnull()) & (dfkillers['pov rate'].notnull())]
notnullkillers.to_csv("..\\data\\cleaned\\notnull_df.csv")

### Cambio del tipo de los valores del df (de str a int/float)

In [ ]:
dfkillers.info

(Este proceso desarrollado se encuentra en el notebook [4converttypes.ipynb](\\notebooks\\4converttypes.ipynb).)

Los valores de la tabla son *object* aun siendo numéricos, por lo que hay que transformarlos.

Sin embargo, hay otros caracteres en la celda, como "11-16" o "203,456.000", por lo que hay que limpiarlos con ```split()``` y ```replace()```:

In [ ]:
df[df['proven victims'].str.contains('–')]

+ Conservamos solamente el primer número de víctimas [0], es más seguro que el segundo (sin confirmar)

In [ ]:
df['proven victims']=df['proven victims'].str.split('–').str[0]
#conservamos solamente el primer número de víctimas [0], es más seguro que el segundo (sin confirmar)

+ Eliminamos las comas de los millares

In [ ]:
df['density']=df['density'].str.replace(",","")

+ Ya se puede transformar los datos a int o float, según corresponda

In [ ]:
df['proven victims']=df['proven victims'].astype(int)
df['pov rate']=df['pov rate'].astype(float)
df['density']=df['density'].astype(float)

# Análisis de los datos y comprobación de hipótesis

(Este proceso desarrollado se encuentra en el *notebook* [5stats.ipynb](\\notebooks\\5stats.ipynb).)
+ Creación de nuevas series y dataframes a partir de los datos
+ Obtención de datos estadísticos básicos por columnas (```mean(), max(), median(), min(), mode(),describe(include='all')```)

+ Obtención del número de asesinos que hay en cada país para que el análisis sea más exacto

In [ ]:
df['country'].value_counts(ascending=False)
#box plot with quartiles

+ Se crea un nuevo dataframe con esta información, que luego uniremos al df principal con ```merge()```.

In [ ]:
#Nº DE ASESINOS POR PAÍS
killerspercountry=df.groupby('country')['name'].count().sort_values(ascending=False).reset_index()
killerspercountry.rename(columns={'name':'amount'},inplace=True)

In [ ]:
killerspercountry['amount'].agg(['median','mean','max','min'])
#98 es un outlier q descompensa la media

+ Se une la nueva columna de asesinos por país al df principal con ```merge()```.

In [ ]:
dfamount=df.merge(killerspercountry,on='country',how='left')
dfamount.head()

In [ ]:
#pov rate and country based on number of victims
dfamount.groupby('proven victims')[['pov rate','country','amount']].agg({'pov rate':['mean','max','min'],'amount':['mean','max','min'],'country': 'first'}).reset_index().sort_values(by=('proven victims'),ascending=False)

In [ ]:
killerspercountrypovrdens=dfamount.groupby('country')[['density','pov rate','amount']].agg({'density':'first','pov rate':'first','amount':'first'}).round(2).sort_values(by='amount', ascending=False).reset_index()
killerspercountrypovrdens
#this shows the amount of killers per country related to its density and pov rate (and victims)

### Hipótesis 3
1. Países con alta densidad de población

In [ ]:
#higher density countries vs victims
df[df['density']> df['density'].median()].sort_values(by='proven victims',ascending=False)

In [ ]:
victimdensitycountry=df.groupby(['proven victims','country'])['density'].agg(['first']).round(2).sort_values(by='first', ascending=False).reset_index()
victimdensitycountry
# victimsdensity

Repetimos este último df teniendo en cuenta el número de asesinos que hay en cada país:

In [ ]:
#pov rate and country based on number of victims
dfamount.groupby('proven victims')[['pov rate','country','amount']].agg({'pov rate':['mean','max','min'],'amount':['mean','max','min'],'country': 'first'}).reset_index().sort_values(by=('proven victims'),ascending=False)

In [ ]:
dfamount['density'].median()

In [ ]:
killerspercountrypovrdens=dfamount.groupby('country')[['density','pov rate','amount']].agg({'density':'first','pov rate':'first','amount':'first'}).round(2).sort_values(by='amount', ascending=False).reset_index()
killerspercountrypovrdens
#this shows the amount of killers per country related to its density and pov rate (and victims)

+ El df creado arriba muestra la densidad media, máxima y mínima según el país y el número de víctimas.
Con él, se desmiente mi primera hipótesis, parece que la densidad de población no hace que los asesinos sean más 'prolíficos', aunque sí que están presentes en esos lugares.

+ Aun así, lugares como Rusia tienen una baja densidad de población a nivel de país, pero probablemente las ciudades concretas donde actuaron tuvieran un valor mayor.

### Hipótesis 2
2. Clases socioeconómicas bajas

In [ ]:
df['country'][df['pov rate'].idxmax()] #costa rica tiene el mayor poverty rate (de los q hay info)

In [ ]:
df['country'][df['pov rate'].idxmin()] #Rep Checa tiene el menor poverty rate (de los q hay info)

In [ ]:
df['country'][df['pov rate']>df['pov rate'].median()].unique()
#países con un pov rate por encima de la mediana del dataset

In [ ]:
df['country'][df['pov rate']<df['pov rate'].median()].unique()
#países con un pov rate inferior a la mediana del dataset

In [ ]:
#pov rate and country based on number of victims
df.groupby('proven victims')[['pov rate','country']].agg({'pov rate':['mean','max','min'],'country': 'first'}).reset_index().sort_values(by=('proven victims'),ascending=False)

### Hipótesis 4
¿Realmente son más comunes en América y Europa?

In [ ]:
(df['country']== 'United States').value_counts() #98 from USA out of 355
#pie chart
df['country'].value_counts(ascending=False)
#box plot with quartiles

>Parece que, efectivamente, en Europa y el norte de Asia hay muchos más casos, mientras que, al menos documentados, hay muy poco en países como Egipto o Yemen.
>
>Sí que sorprende la predominancia de Rusia frente a todo menos EE.UU., que se lleva la palma clarísimamente.
>
>Países como Colombia o Venezuela, que tenían una de las mayores medias de asesinatos, parecen deberse a una sola persona o pocas de ellas, no a múltiples asesinos.


> Habrá que hacer un scatter plot para diferenciar si un país tiene muchos asesinatos y pocos asesinos o viceversa (como Colombia con garavito (outlier)

### Hipótesis 1
+ En que países hay más asesinos en serie y víctimas

In [ ]:
#pov rate and country based on number of victims
df.groupby('proven victims')[['pov rate','country']].agg({'pov rate':['mean','max','min'],'country': 'first'}).reset_index().sort_values(by=('proven victims'),ascending=False)

In [ ]:
#higher density countries vs victims
df[df['density']> df['density'].mean()].sort_values(by='proven victims',ascending=False)

In [ ]:
maxvicpercountry=df.groupby('country')['proven victims'].max().sort_values(ascending=False)
maxvicpercountry

In [ ]:
df.groupby('country')['proven victims'].mean().round(2).sort_values(ascending=False)


In [ ]:
df.groupby('country')['proven victims'].sum().idxmax()#.sort_values(ascending=False)

Garavito es un outlier, descompensa los datos de colombia y venezuela

> Parece que en países latinoamericanos hay + asesinatos de media y tb de mayor nº por asesino (asesinos más 'prolíficos').
>
> Sin embargo, el país donde más asesinos ha habido es EE.UU (y si se suman todos los asesinos por país, es el país donde más crímenes se han cometido).

In [ ]:
victimstats=df.groupby('country')['proven victims'].agg(['median','mean','max','min']).round(1).sort_values(by='median', ascending=False)
victimstats.reset_index(inplace=True)

### Hipótesis 5
+ En que países los descubren o capturan antes

# Visualizaciones

(Este proceso desarrollado se encuentra en el *notebook* [6visuals.ipynb](\\notebooks\\6visuals.ipynb).)
+ A continuación se tratará de comprobar o refutar las hipótesis planteadas por medio de gráficas que respalden los datos.

### Hipótesis 2 y 3. Países con alta densidad de población e índice de pobreza

In [ ]:
killerspercountrypovrdens=df.groupby('country')[['density','pov rate','amount']].agg({'density':'first','pov rate':'first','amount':'first'}).round(2).sort_values(by='amount', ascending=False).reset_index()
killerspercountrypovrdens
#this shows the amount of killers per country related to its density and pov rate

In [ ]:
#Tomamos la densidad de Singapur como un outlier, ya que descompensa todas las medidas y scatterplots
df['country'][df['density'].idxmax()]

+ Se hace una matriz de correlación con todas las variables numéricas:

In [ ]:
#correlation matrix (same for pov rate not null)
dfcorr=(df[['proven victims','pov rate','density','amount']][(df['pov rate'].notnull())&(df['density'].notnull()) ]).corr()
dfcorr

Hay una clara correlación (0.75) entre el índice de pobreza y el número de asesinos en serie de un país, por lo que lo comprobaremos en un mapa de calor.

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(dfcorr,
            vmin=-1,
            vmax=1,
            cmap=sns.diverging_palette(0, 20, s=95, l=15, n=7),
            square=False,
            annot=True,
            xticklabels=['Victims','Poverty rate','Pop density', 'Nr of killers'],
            yticklabels=['Victims','Poverty rate','Pop density', 'Nr of killers']
            );
plt.title('Serial killer demographic heatmap')


+ Parece que hay una correlación positiva entre el índice de pobreza y el número de asesinos de un país.

+ Por otra parte, hay una correlación negativa débil entre el la densidad y el número de asesinos (así como entre el índice de pobreza y la densidad, aunque eso no afecta al estudio).

El scatterplot no permite verlo con tanta claridad (aunque he recortado los límites para descartar outliers de densidad, como Singapur, y de víctimas, como Colombia):

+ Densidad frente a número de asesinos y de víctimas:

In [ ]:
plt.subplot(1,2,1)
sns.scatterplot(x='amount', y='density', data=df[df['density'].notnull()],color='darkred')
plt.xlim(0,30)
plt.ylim(0,550)

plt.subplot(1,2,2)
sns.scatterplot(x='proven victims', y='density', color='darksalmon',data=df[df['density'].notnull()])
plt.xlim(0,140)
plt.ylim(0,550)

+ Índice de pobreza frente a número de asesinos y de víctimas:

In [ ]:
scattpov=df[df['pov rate'].notnull()]
scattpov.drop_duplicates(subset='country',inplace=True)

plt.subplot(1,2,1)
sns.scatterplot(x='amount', y='pov rate', data=scattpov,color='darkred')
plt.xlim(0,15)
plt.ylim(0.05,0.225)

plt.subplot(1,2,2)
sns.scatterplot(x='proven victims', y='pov rate', color='darksalmon',data=scattpov)
plt.xlim(0,70)
plt.ylim(0.05,0.225)

Tomamos EE.UU. como un outlier en cuanto a número de asesinos por país, así que lo excluímos de la gráfica.

### Hipótesis 4. ¿Son más comunes en América y Europa?

In [ ]:
df['country'].mode()

In [ ]:
df['country'].value_counts(ascending=False)
#box plot with quartiles

In [ ]:
topcountries=df[df['amount']>18]
other=df[df['amount']<=18]
other['country']='Other'
other.drop_duplicates('country',inplace=True)
piecountries=pd.concat([topcountries,other])
piecountries.drop_duplicates(['country','amount'],inplace=True)

Parece que hay unos cuantos países que predominan en cuanto al número de asesinos. Haré un pie chart que refleje los más frecuentes:

In [ ]:
plt.figure(figsize=(8,8))
plt.pie((piecountries['amount']),
        labels=piecountries['country'],
        autopct='%1.2f%%',
        colors=['tomato','darkred','lightsalmon','brown','maroon'])

plt.title('Distribution of Serial Killers by Country\n(Top Countries Only)');

También podemos ver la comparación de todo el dataset en relación con EE.UU.:

In [ ]:
plt.figure(figsize=(8,8))
plt.pie((df['country']== 'United States').value_counts(),
        labels=['Other','United States'],
        autopct='%1.2f%%',
        colors=['maroon','tomato'])

plt.title('Percentage of Killers in the U.S. in relation to other countries');

En definitiva, EE.UU. tiene muchísimos más asesinos en serie a lo largo de los años.

Sin embargo, países como Rusia, Ucrania o Sudáfrica le pisan los talones, ya que entre todos sus asesinos componen cerca del 96% del dataset.

Por lo tanto, ante la pregunta de si los asesinos en serie son más frecuentes en EE.UU. y Europa, podemos decir que sí, en EE.UU. y en Europa del este y Asia occidental.

### Hipótesis 1. ¿En qué países cometen más asesinatos o hay más asesinos en serie?

In [ ]:
killerspercountry=df.groupby('country')['name'].count().sort_values(ascending=False).reset_index()

+ Haré un bar plot para comparar la cantidad de asesinos por país. (Descartaré los países con menos de dos asesinos para no saturar el eje de la gráfica).

In [ ]:
distplot4=df[['country','amount']][df['amount']>4].sort_values(by='amount',ascending=False)

In [ ]:
#HISTOGRAMA
plt.figure(figsize=(6,10));
sns.displot(distplot4['country'],color='darkred');
plt.axis('tight');
plt.xticks(rotation=90);
plt.xlabel('Country');
plt.ylabel('Number of killers');
plt.title('Number of Killers per Country');
plt.yticks(range(0,101,25))

Ya sabemos que en EE.UU. están la mayoría, por lo que nos acercaremos a los demás para verlos con más detalle.

In [ ]:
#repeat with more zoom
plt.figure(figsize=(5,9))
sns.displot(distplot4['country'],color='darkred')
plt.axis('tight')
plt.xticks(rotation=90);
plt.ylim(4,30)
plt.yticks(range(5,30,5))

plt.xlabel('Country');
plt.ylabel('Number of killers');
plt.title('Number of Killers per Country (Detail)');

A lo largo del análisis me surgió la duda de **si en ciertos países había muchas víctimas porque había muchos asesinos o porque había un asesino más "prolífico" o activo**.

Por ello, decidí hacer un boxplot para identificar estos valores y posibles outliers.

In [ ]:
df['amount'].agg(['median','mean','max','min'])
#98 es un outlier q descompensa la media

In [ ]:
df[df['proven victims']>30].sort_values(by='proven victims',ascending=False)

In [ ]:
plt.figure(figsize=(8,8))
sns.boxplot(data=(df[df['proven victims']>30].sort_values(by='proven victims',ascending=False)), x='amount',y='proven victims',color='darkred');
# plt.xticks(rotation=90);
plt.ylim(29,140)
plt.yticks(range(30,141,15))
plt.title('Number of Killers per Country and their Victims')
plt.xlabel('Number of killers in a country')
plt.ylabel('Number of victims')

+ Haré también un scatterplot para que quede más clara esta distribución dispar. 

In [ ]:
plt.figure(figsize=(9,6))
sns.scatterplot(data=df[df['proven victims']>40].sort_values(by='country',ascending=True),
                x='proven victims',
                y='amount',
                hue = 'country',
                palette='dark',
                s=100);
plt.ylim(0,100)
plt.yticks([0,5,20,40,80,100])
plt.gca().invert_xaxis()
plt.xticks(range(40,141,20))

plt.title('Number of Killers in a Country and their Victims')
plt.ylabel('Number of killers in a country')
plt.xlabel('Number of victims')
plt.legend(loc = 'upper left', frameon = False, ncol = 2)

+ Como se puede apreciar, hay países como EE.UU. con un gran número de asesinos, pero menos víctimas por asesino.

+ Por otro lado hay asesinos particularmente "prolífico" que hacen que en países como Colombia o Venezuela con tan solo dos asesinos haya un mayor número de víctimas.

In [ ]:
#barplot to show victim trends per country
plt.figure(figsize=(10,6))
sns.barplot(x=maxvicpercountry.values, y=maxvicpercountry.index)

In [ ]:
victimstats=df.groupby('country')['proven victims'].agg(['median','mean','max','min']).round(1).sort_values(by='median', ascending=False)
victimstats.reset_index(inplace=True)

#### Hipótesis 5. ¿Dónde los descubren o capturan antes?

# Conclusiones

Tras este análisis, se pueden sacar varias conclusiones de las hipótesis iniciales:

#### 1.Países en los que estos asesinos están más presentes o donde cometen más crímenes

Más de un cuarto de los asesinos en serie del *dataset* se encuentran en Estados Unidos, seguido de cerca por Rusia, Ucrania y Sudáfrica. Esto puede estar relacionado con la mayor cantidad de estudios y recursos dedicados a la criminología en Estados Unidos desde sus comienzos por parte del FBI. Sin embargo, también se observaron casos notables en países con menos visibilidad, como Colombia, en los que el número de asesinos era mucho menor pero el de víctimas era descomunal.
La disparidad en el número de víctimas por asesino entre diferentes países sugiere que en algunos lugares se da más visibilidad y se publican más datos sobre estos casos que en otros, lo que podría explicar la diferencia en la visibilidad de los asesinos en serie.
#### 2. Clase Socioeconómica

Se encontró una correlación significativa (0.74) entre el número de asesinos en serie y el índice de pobreza. Aunque esta correlación es prometedora, es cierto que había muchos valores nulos respecto a esta variable y que esa medida solamente no basta para determinar la clase social de cada asesino en concreto. Sin embargo, el país mas pobre de mi *dataset* era Ecuador y coincide con el dato de las 140 víctimas por un solo asesino (Luis Garavito), ya que este actuó tanto en Colombia, como en Venezuela y Ecuador. Sería interesante, a partir de esta correlación preliminar, considerar otras varibles para llegar a comprender completamente la relación entre clases socioeconómicas y asesinos en serie.
#### 3. Densidad de Población:

Aunque se observó una correlación negativa (-0.62) entre la densidad de población y el número de asesinos en serie, estos datos son poco concluyentes y su causalidad es cuestionable. Se requiere un análisis más exhaustivo que tenga en cuenta la densidad de población en los núcleos urbanos concretos donde opera cada asesino en serie.
Igualmente, serviría como punto de partida para otro posible análisis, dado que los datos indican que puede haber un nexo de la facilidad de que surja un asesino en serie en zonas poco pobladas.
#### 4.Predominancia de EE.UU. y Europa:

La mayoría de los asesinos en serie en el conjunto de datos provienen de EE.UU. y Europa del Este, lo que puede reflejar una mayor visibilidad y estudio de estos casos en estas regiones. Esto podría atribuirse a la presencia de organismos gubernamentales y recursos para investigar y rastrear este perfil de asesino.
#### 5.Captura de Asesinos:

La tasa de víctimas por asesino varía significativamente entre países. Los países con una alta tasa de víctimas por asesino, como Colombia, sugieren que los asesinos pueden no ser capturados con la misma rapidez debido a la falta de recursos o formación en perfiles psicológicos.

En resumen, este análisis exploratorio de datos sobre asesinos en serie revela tendencias interesantes, pero también destaca la necesidad de investigaciones más detalladas y contextualizadas de cada caso para comprender completamente los factores que subyacen tras de la presencia y el comportamiento de los asesinos en serie en diferentes países.

## Consideraciones para el futuro
+ Hay datos del *dataframe* que no he explotado en el análisis. Aun así, he decidido no borrarlos porque podría profundizarse en ellos en un segundo estudio. Si bien he podido comprobar mis hipótesis con las variables utilizadas, se podrían sacar muchas otras de los datos aún por explotar:

+ Por ejemplo, se podría limpiar la columna de *years active* para determinar patrones de actividad por país o por número de víctimas.
+ Se podría tratar de diferenciar a los asesinos según si son hombres o mujeres, para ver si cambia algún dato demográfico según este factor.
+ También se podría hacer una búsqueda de *keywords* de la columna *notes* para buscar patrones en su *modus operandi*.
+ Por último, se podría dividir a los asesinos por continente para obtener una visión más global de la distribución.